In [1]:
%matplotlib inline
import sys, os, time
# Select GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K
from keras.utils import to_categorical
from keras.applications.vgg19 import VGG19, preprocess_input

Using TensorFlow backend.


## Load data

In [2]:
x_val = np.load("data/x_val.npy") # loaded as RGB
x_val = preprocess_input(x_val) # converted to BGR

In [3]:
y_val = np.load("data/y_val.npy")

In [4]:
y_val_one_hot = to_categorical(y_val, 1000)

In [5]:
keras_idx_to_name = {}
f = open("data/synset_words.txt","r")
idx = 0
for line in f:
    parts = line.split(" ")
    keras_idx_to_name[idx] = " ".join(parts[1:])
    idx += 1
f.close()

## Benchmark models

In [6]:
def top_k_accuracy(y_true, y_pred, k=1):
    '''From: https://github.com/chainer/chainer/issues/606
    
    Expects both y_true and y_pred to be one-hot encoded.
    '''
    argsorted_y = np.argsort(y_pred)[:,-k:]
    return np.any(argsorted_y.T == y_true.argmax(axis=1), axis=0).mean()

In [7]:
K.clear_session()
model = VGG19()

In [8]:
y_pred = model.predict(x_val, verbose=1)

50000/50000 [==============================] - 205s 4ms/step


#### Top-1 Accuracy

Compare to 0.713 from Keras documentation

In [9]:
top_k_accuracy(y_val_one_hot, y_pred, k=1)

0.71256

#### Top-5 Accuracy

Compare to 0.900 from Keras documentation

In [10]:
top_k_accuracy(y_val_one_hot, y_pred, k=5)

0.89988